In [8]:
# 프레임워크: OpenCV
# 머신러닝 알고리즘: KNN

import cv2 # 웹캠 제어 및 ML 사용
import mediapipe as mp
import numpy as np
from PIL import ImageFont, ImageDraw, Image
from sklearn.preprocessing import LabelEncoder
from collections import deque
import time


consonant_labels = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
vowel_labels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ','ㅣ','ㅘ','ㅚ', 'ㅙ', 'ㅝ','ㅟ','ㅞ','ㅢ']
command_labels = ['shift', 'space', 'end'] # '쌍자음 만들기' 명령어만 남김
labels = consonant_labels + vowel_labels + command_labels

#font_path = "/Users/hwi/Library/Fonts/GowunDodum-Regular.ttf"
font_path = "C:/Windows/Fonts/hmfmmuex.ttc"
dataset_file = 'combined_hand_landmark.txt'

history = deque(maxlen=5)


# 인덱스를 라벨로 변환하기 위한 맵 생성
label_map = {i: label for i, label in enumerate(labels)}


def putText_korean(image, text, pos, font_path, font_size, color):
    """
    한글 텍스트를 화면에 표시.
    """
    # OpenCV 이미지를 PIL 이미지로 변환
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    # 폰트 로드
    font = ImageFont.truetype(font_path, font_size)
    # 텍스트 그리기
    draw.text(pos, text, font=font, fill=tuple(color[::-1]))
    # PIL 이미지를 다시 OpenCV 이미지로 변환하여 반환
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# >> mediapipe Hands 모델 로드 <<
mp_hands = mp.solutions.hands # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티
mp_drawing = mp.solutions.drawing_utils # 웹캠 영상에서 손가락 마디와 포인트를 그릴 수 있게 도와주는 유틸리티


# >> mediapipe Hands 모델 초기화 <<
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


# >> 각도 계산 <<
def calculate_angles(joint):
    """
    랜드마크 위치를 사용하여 관절 간의 각도를 계산.
    joint가 (21,3) ndarray 형식으로 들어와서,
    (x, y, z) 랜드마크 21개를 기반으로 15개 관절 각도를 반환.
    """
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1

    # 벡터 정규화
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 내적과 arccos를 사용해 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n',
        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    
    # 라디안을 각도로 변환
    angle = np.degrees(angle)

    return angle.astype(np.float32)

    

# >> 데이터 로드 & 전처리 <<
def load_and_preprocess(dataset_file):
    labels = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols = 0, encoding ="EUC-KR", dtype = str)
    angles = np.genfromtxt(dataset_file, delimiter=',', skip_header=1,usecols= range(1,127), encoding = "EUC-KR").astype(np.float32) # **각 제스처들의 라벨과 각도가 저장되어 있음, 정확도를 높이고 싶으면 데이터를 추가해보자!**

    all_angles = []
    for row in angles:
        lh_landmarks = row[:63].reshape(21,3) # 총 21개의 landmark, 각각 총 3개의 (x,y,z) 좌표
        rh_landmarks = row[63:].reshape(21,3)

        lh_angles = calculate_angles(lh_landmarks) if np.any(lh_landmarks) else np.zeros(15)
        rh_angles = calculate_angles(rh_landmarks) if np.any(rh_landmarks) else np.zeros(15)

        all_angles.append(np.concatenate([lh_angles, rh_angles]))
        
    all_angles = np.array(all_angles, dtype = np.float32)

    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    

    return all_angles, encoded_labels, encoder



# >> 모델 학습 << 
def train_model(dataset_file):
    """
    수집한 데이터를 knn 모델로 학습.
    """
    # 전처리한 결과 로드
    X, y, encoder = load_and_preprocess(dataset_file)
    
    # knn 모델 생성
    knn = cv2.ml.KNearest_create()
    print("--- knn 모델을 생성 & 학습 시작 ---")
    
    # 모델 학습
    knn.train(X, cv2.ml.ROW_SAMPLE, y.astype(np.int32))
    print("\n--- 학습 완료 ---")
    print(f"총 {X.shape[0]}개의 샘플 데이터로 모델을 학습했습니다.")
    print(f"데이터의 특징(feature) 차원으로 추출한 관절 각도의 개수: {X.shape[1]}")
    return knn, encoder


# >> 실시간 프레임 처리 <<

def run_frame(trained_knn_model, encoder):
    """
    학습된 knn 모델을 사용해서 웹캠에서 실시간으로 제스처 인식.
    """
    global histroy
    
    if trained_knn_model is None:
        print("학습된 모델이 없어 검증을 시작할 수 없습니다.")
        return

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("오류: 웹캠을 열 수 없습니다.")
        return

    print("웹캠이 활성화되었습니다. 'q'를 눌러 종료하세요.")
    
    entered_string = []
    display_label = ""
    display_start_time = None
    display_duration = 2 # 해당 label을 웹캠 화면에 표시할 시간(초)
    

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("프레임을 찾을 수 없습니다.")
            break

        # 프레임을 수평으로, rgb로 변환
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 손 정보를 처리
        result = hands.process(frame_rgb)

        # 다시 brg로 변환
        #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # 변수 초기화
        predicted_label = "" # 예측 할 때마다 초기화
        guide_text = "손을 보여주세요"

        # 손 개수가 감지되었는지 확인
        if result.multi_hand_landmarks:
            left_hand, right_hand = None, None

            for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
                handedness_label = result.multi_handedness[i].classification[0].label
                joint = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])

                if handedness_label == "Left":
                    left_hand = calculate_angles(joint)
                elif handedness_label == "Right":
                    right_hand = calculate_angles(joint)

            if left_hand is None:
                left_hand = np.zeros(15)
            if right_hand is None:
                right_hand = np.zeros(15)

            feature_vector = np.concatenate([left_hand, right_hand]).reshape(1,-1).astype(np.float32)

            if not np.all(np.isnan(feature_vector)): # 아무 것도 인식되지 않는게 아니라면(인식되는 손이 있다면)
                try:
                    # ret, results, neighbours, dist
                    _, results, _, _ = trained_knn_model.findNearest(feature_vector, k=3)
                    predicted_label = encoder.inverse_transform([int(results[0][0])])[0]
                    predicted_text = f"{'왼손' if handedness_label == 'Left' else '오른손'}: {predicted_label}"

                    # 5번 연속으로 같은 동작으로 인식되면 해당 동작을 인식
                    history.append(predicted_label)
                    if len(history) == 5 and len(set(history)) == 1:  # 최근 5개가 동일하면
                        entered_string.append(predicted_label)
                        print(predicted_label)
                        history.clear()

                        # 인식된 동작을 웹캠 화면에 표시
                        display_label = predicted_label
                        display_start_time = time.time()
                            
                except Exception as e:
                    guide_text = "알 수 없는 제스처"
            else:
                guide_text = "손이 인식되지 않습니다."
            # 화면에 랜드마크 그리기
            for res in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, res, mp_hands.HAND_CONNECTIONS)

        # 웹캠 화면에 띄울 텍스트
        # 제스처 인식되면 일정 시간 동안 해당 문자를, 그렇지 않다면 안내 문구를 표시
        if display_start_time and ((time.time()-display_start_time) < display_duration):
            display_text = display_label
        else:
            display_text = guide_text

        
        # 텍스트 위치 계산 및 표시
        text_size = 50 # 폰트 크기
        font = ImageFont.truetype(font_path, text_size)
        bbox = font.getbbox(guide_text)  # (xmin, ymin, xmax, ymax)
        text_w = bbox[2] - bbox[0]
        text_h = bbox[3] - bbox[1]
            
        text_x = int((frame.shape[1] - text_w) / 2)
        text_y = int(frame.shape[0] - 50)
            
        frame = putText_korean(frame, display_text, (text_x, text_y), font_path, text_size, (0, 255, 0))
            
        cv2.imshow('frame', frame)
            
        if cv2.waitKey(1) == ord('q'):
            break
    
    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    return entered_string
            
            

if __name__ == "__main__":
    dataset_file = 'combined_hand_landmark.txt'
    
    knn, encoder = train_model(dataset_file)
    
    input_string = run_frame(knn, encoder)

    print("\n--- 최종 인식된 제스처 목록 ---")
    print(input_string)

I0000 00:00:1759043745.034221 59938343 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1759043745.061238 60007289 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759043745.069761 60007289 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


--- knn 모델을 생성 & 학습 시작 ---

--- 학습 완료 ---
총 1364개의 샘플 데이터로 모델을 학습했습니다.
데이터의 특징(feature) 차원으로 추출한 관절 각도의 개수: 30


2025-09-28 16:15:46.189 python3.9[73984:59938343] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


웹캠이 활성화되었습니다. 'q'를 눌러 종료하세요.


W0000 00:00:1759043749.705496 60007289 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


ㅜ
ㅜ
ㅜ
ㅜ
ㅜ
ㅜ
space
ㅕ
ㅗ
space

--- 최종 인식된 제스처 목록 ---
['ㅜ', 'ㅜ', 'ㅜ', 'ㅜ', 'ㅜ', 'ㅜ', 'space', 'ㅕ', 'ㅗ', 'space']


In [7]:
# >> K값 변화에 따른 모델 정확도 확인 <<

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import cv2
from sklearn.preprocessing import LabelEncoder


def calculate_angles(joint):
    """
    랜드마크 위치를 사용하여 관절 간의 각도를 계산.
    joint가 (21,3) ndarray 형식으로 들어와서,
    (x, y, z) 랜드마크 21개를 기반으로 15개 관절 각도를 반환.
    """
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1

    # 벡터 정규화
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 내적과 arccos를 사용해 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n',
        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    
    # 라디안을 각도로 변환
    angle = np.degrees(angle)

    return angle.astype(np.float32)


# >> 데이터 로드 & 전처리 <<
def load_and_preprocess(dataset_file):
    labels = np.genfromtxt(dataset_file, delimiter=',', skip_header=1, usecols = 0, encoding ="EUC-KR", dtype = str)
    angles = np.genfromtxt(dataset_file, delimiter=',', skip_header=1,usecols= range(1,127), encoding = "EUC-KR").astype(np.float32) # **각 제스처들의 라벨과 각도가 저장되어 있음, 정확도를 높이고 싶으면 데이터를 추가해보자!**

    all_angles = []
    for row in angles:
        lh_landmarks = row[:63].reshape(21,3) # 총 21개의 landmark, 각각 총 3개의 (x,y,z) 좌표
        rh_landmarks = row[63:].reshape(21,3)

        lh_angles = calculate_angles(lh_landmarks) if np.any(lh_landmarks) else np.zeros(15)
        rh_angles = calculate_angles(rh_landmarks) if np.any(rh_landmarks) else np.zeros(15)

        all_angles.append(np.concatenate([lh_angles, rh_angles]))
        
    all_angles = np.array(all_angles, dtype = np.float32)

    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    

    return all_angles, encoded_labels, encoder




def evaluate_knn_accuracy(dataset_file):
    """
    1부터 10까지의 k 값에 따른 KNN 모델의 정확도를 평가합니다.
    """
    X, y, encoder = load_and_preprocess(dataset_file)
    
    # 훈련 세트와 테스트 세트로 분할 (8:2 비율)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    accuracies = []
    
    # k 값을 1부터 10까지 반복
    for k in range(1, 11):
        print(f"\n--- k = {k} ---")
        knn = cv2.ml.KNearest_create()
        knn.setDefaultK(k)
        
        # 모델 학습
        knn.train(X_train, cv2.ml.ROW_SAMPLE, y_train.astype(np.int32))
        
        # 테스트 데이터로 예측
        # k=k를 명시하여 현재 k값을 사용하도록 함
        _, results, _, _ = knn.findNearest(X_test, k=k)
        
        # 예측된 라벨과 실제 라벨 비교
        y_pred = results.flatten()
        
        # 정확도 계산
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append((k, accuracy))
        print(f"정확도: {accuracy:.4f}")

    return accuracies

if __name__ == "__main__":

    dataset_file = 'combined_hand_landmark.txt'

    # k 값에 따른 정확도 평가
    print("--- k 값에 따른 정확도 평가를 시작합니다. ---")
    k_accuracies = evaluate_knn_accuracy(dataset_file)
    
    print("\n--- K 값별 최종 정확도 ---")
    for k, acc in k_accuracies:
        print(f"K = {k}: 정확도 = {acc:.4f}")
    
    # 여기서 최적의 k 값을 선택하여 실제 모델을 학습하고 run_frame 실행
    # best_k = max(k_accuracies, key=lambda item: item[1])[0]
    # print(f"\n최적의 K 값은 {best_k} 입니다.")
    
    # 최적의 k 값으로 최종 모델 학습 및 실시간 제스처 인식
    # knn, encoder = train_model(dataset_file)
    # input_string = run_frame(knn, encoder)
    # print("\n--- 최종 인식된 제스처 목록 ---")
    # print(input_string)

--- k 값에 따른 정확도 평가를 시작합니다. ---

--- k = 1 ---
정확도: 0.9560

--- k = 2 ---
정확도: 0.9194

--- k = 3 ---
정확도: 0.9194

--- k = 4 ---
정확도: 0.8974

--- k = 5 ---
정확도: 0.8938

--- k = 6 ---
정확도: 0.8828

--- k = 7 ---
정확도: 0.8938

--- k = 8 ---
정확도: 0.8791

--- k = 9 ---
정확도: 0.8828

--- k = 10 ---
정확도: 0.8645

--- K 값별 최종 정확도 ---
K = 1: 정확도 = 0.9560
K = 2: 정확도 = 0.9194
K = 3: 정확도 = 0.9194
K = 4: 정확도 = 0.8974
K = 5: 정확도 = 0.8938
K = 6: 정확도 = 0.8828
K = 7: 정확도 = 0.8938
K = 8: 정확도 = 0.8791
K = 9: 정확도 = 0.8828
K = 10: 정확도 = 0.8645
